# Produce False Colour Geotiffs Iteratively

**What does this notebook do?** 


**Date** March 2018

**Author** Claire Krause

**Tags** 

## This is the bit that allows you to call in a stand alone python file. 

The two python files `DEAPlotting.py` and `DEADataHandling.py` are both sitting in the `algorithms` folder.

In [1]:
% pylab notebook
% load_ext autoreload
% autoreload

from datacube import Datacube
from datetime import datetime
from IPython.display import clear_output

import sys
import os
sys.path.append(os.path.abspath('/g/data/r78/cek156/dea-notebooks/algorithms'))
import DEAPlotting
import DEADataHandling

dc = Datacube(app = 'test')

Populating the interactive namespace from numpy and matplotlib


## Extract some data using our imported function

In [2]:
AllSensors = ['ls5', 'ls7', 'ls8']

for Sensor in AllSensors:
    print('Working on {0}'.format(Sensor))
    for year in range(1985,2018,2):
        try:
            print('Working on {0} to {1}'.format(year, year + 1))
            queryls = {'lat': (-15.357768, -15.915976),
                       'lon': (128.581927, 128.980604),
                       'time': ('{0}-01-01'.format(year), '{0}-12-30'.format(year + 1))
                       }
            querys2 = {'output_crs': 'EPSG:3577',
                       'resolution': (-10, 10),
                       'lat': (-15.357768, -15.915976),
                       'lon': (128.581927, 128.980604),
                       'time': ('{0}-01-01'.format(year), '{0}-12-30'.format(year + 1))
                       }

            Alldata, LScrs, LSaffine = DEADataHandling.load_nbarx(dc, Sensor, queryls, 
                                                                  bands_of_interest = ['swir1','nir','green'], 
                                                                  product = 'nbar')

            ## We only want to plot up images that have no more than 10% missing pixels
            acceptable_missing_data = 0.1
            threshold90 = Alldata.x.count() * Alldata.y.count() * acceptable_missing_data
            Alldata = Alldata.where(Alldata.green.isnull().sum(axis = (1, 2)) <= int(threshold90)).dropna(dim = 'time')

            ## Draw a false colour image using our imported function and save
            for ix, times in enumerate(Alldata.time):
                outnameGTiff = '/g/data/r78/Kununurra/{0}_{1}.tif'.format(str(times.time.values)[:10], Sensor)
                outnamejpg = '/g/data/r78/Kununurra/{0}_{1}.jpg'.format(str(times.time.values)[:10], Sensor)
                plt, fig = DEAPlotting.three_band_image(Alldata.isel(time = ix), ['swir1','nir','green'], title = '')   
                DEADataHandling.dataset_to_geotiff(outnameGTiff, Alldata.isel(time = ix))
                savefig(outnamejpg, dpi = 300)
                plt.close("all")
                close(fig)
        except TypeError:
            print('No data available for this time step')
            pass
        clear_output()